In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import numpy as np
import sys
import os
from tqdm import tqdm
import pickle

# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from src.ParObsSnakeEnv import ParObsSnakeEnv
from src.FullObsSnakeEnv import FullObsSnakeEnv
from src.utils import compute_metrics

In [7]:
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super(DQN, self).__init__()
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(input_dim, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.fc3 = nn.Linear(self.hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
# Define the DQN agent
class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, memory_size=10000, batch_size=64):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size

        self.memory = deque(maxlen=memory_size)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = DQN(state_dim, action_dim).to(self.device)
        self.target_model = DQN(state_dim, action_dim).to(self.device)
        self.target_model.load_state_dict(self.model.state_dict())
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.criterion = nn.MSELoss()

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1), None
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        state = state.reshape((1, -1))
        with torch.no_grad():
            # print(">>>>", state.shape)
            q_values = self.model(state)
        return torch.argmax(q_values).item(), None

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)

        states = states.reshape((self.batch_size, -1))
        next_states = next_states.reshape((self.batch_size, -1))

        # Compute current Q-values
        q_values = self.model(states)
        q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

        # Compute target Q-values
        next_q_values = self.target_model(next_states).max(1)[0]
        target_q_values = rewards + (1 - dones) * self.gamma * next_q_values

        # Update the Q-network
        loss = self.criterion(q_values, target_q_values)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Decay epsilon
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def save(self, filename):
        """Saves the entire agent to a file."""
        state = {
            'model_state_dict': self.model.state_dict(),
            'target_model_state_dict': self.target_model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'hyperparameters': {
                'state_dim': self.state_dim,
                'action_dim': self.action_dim,
                'gamma': self.gamma,
                'epsilon': self.epsilon,
                'epsilon_decay': self.epsilon_decay,
                'epsilon_min': self.epsilon_min,
                'batch_size': self.batch_size,
            },
            'memory': list(self.memory),  # Convert deque to list
        }
        with open(filename, 'wb') as f:
            pickle.dump(state, f)
        print(f"Agent saved to {filename}")
    
    @classmethod
    def load(cls, filename, lr=0.001):
        """Loads the agent from a file."""
        with open(filename, 'rb') as f:
            state = pickle.load(f)
        
        # Recreate the agent
        agent = cls(
            state['hyperparameters']['state_dim'],
            state['hyperparameters']['action_dim'],
            lr=lr,
            gamma=state['hyperparameters']['gamma'],
            epsilon=state['hyperparameters']['epsilon'],
            epsilon_decay=state['hyperparameters']['epsilon_decay'],
            epsilon_min=state['hyperparameters']['epsilon_min'],
            batch_size=state['hyperparameters']['batch_size'],
        )
        # Restore the agent's state
        agent.model.load_state_dict(state['model_state_dict'])
        agent.target_model.load_state_dict(state['target_model_state_dict'])
        agent.optimizer.load_state_dict(state['optimizer_state_dict'])
        agent.memory = deque(state['memory'], maxlen=len(state['memory']))
        print(f"Agent loaded from {filename}")
        return agent

    # Train the agent
    def train(self, env, episodes=1000, update_target_every=10):
        for episode in tqdm(range(episodes), desc="Training", unit='episode'):
            state = env.reset()
            total_reward = 0
            done = False

            while not done:
                action, _ = self.choose_action(state)
                next_state, reward, done, _ = env.step(action)
                self.remember(state, action, reward, next_state, done)
                self.replay()
                state = next_state
                total_reward += reward

            
            # print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.3f}")

            # if (episode + 1) % update_target_every == 0:
                self.update_target_model()
            #     print("Updated target model.")

In [9]:
grid_size = 10
# env = FullObsSnakeEnv(grid_size=grid_size, interact=False)
env = ParObsSnakeEnv(grid_size=grid_size, interact=False)
if isinstance(env, FullObsSnakeEnv):
    state_dim = env.observation_space.shape[0] * env.observation_space.shape[1] * env.observation_space.shape[2]
else:
    state_dim = env.observation_space.shape[0]

action_dim = env.action_space.n
agent = DQNAgent(state_dim, action_dim)

In [10]:
num_episodes = 300
agent.train(env, episodes=300)

Training: 100%|██████████| 300/300 [01:17<00:00,  3.88episode/s]


In [11]:
environment = 'full 'if isinstance(env, FullObsSnakeEnv) else 'par'

agent_name = f'dqn_agent_{environment}_{num_episodes}_{grid_size}.pkl'
model_weights_dir = os.path.join('../..', 'models', 'dqn')
os.makedirs(model_weights_dir, exist_ok=True)
agent_path = os.path.join(model_weights_dir, agent_name)

agent.save(agent_path)
# agent = DQNAgent.load(agent_path)

Agent saved to ../../models/dqn/dqn_agent_par_300_10.pkl


In [12]:
if isinstance(env, ParObsSnakeEnv):
    env = ParObsSnakeEnv(grid_size=2*grid_size, interact=False)

num_simulations = 100
metrics_name = f'dqn_metrics_{environment}_{num_episodes}_{env.grid_size}_{num_simulations}.jsn'
model_metrics_dir = os.path.join('../..', 'artifacts', 'models_stats', 'dqn')
os.makedirs(model_metrics_dir, exist_ok=True)
metrics_path = os.path.join(model_metrics_dir, metrics_name)

compute_metrics(agent, env, metrics_path, num_simulations=num_simulations)

  1%|          | 1/100 [00:00<00:12,  7.78it/s]

Snake length: 39, Episode reward: 3272
Snake length: 9, Episode reward: 595
Snake length: 17, Episode reward: 1292


  4%|▍         | 4/100 [00:00<00:06, 15.08it/s]

Snake length: 33, Episode reward: 2728


  6%|▌         | 6/100 [00:00<00:06, 14.45it/s]

...

 98%|█████████▊| 98/100 [00:09<00:00, 12.08it/s]

Snake length: 41, Episode reward: 3436
Snake length: 16, Episode reward: 1220
Snake length: 16, Episode reward: 1276


100%|██████████| 100/100 [00:09<00:00, 10.93it/s]

Snake length: 34, Episode reward: 2846
Snake length: 11, Episode reward: 798


{'snake_lengths': [39,
  9,
  17,
  33,
  27,
  20,
  46,
  23,
  32,
  28,
  33,
  36,
  43,
  35,
  42,
  29,
  32,
  28,
  36,
  14,
  5,
  27,
  17,
  12,
  25,
  27,
  18,
  22,
  24,
  18,
  28,
  29,
  48,
  36,
  24,
  15,
  13,
  36,
  51,
  24,
  33,
  35,
  25,
  38,
  24,
  39,
  19,
  27,
  32,
  38,
  39,
  23,
  23,
  42,
  15,
  48,
  12,
  55,
  20,
  39,
  48,
  27,
  32,
  32,
  22,
  46,
  29,
  21,
  29,
  26,
  22,
  29,
  33,
  34,
  34,
  24,
  42,
  33,
  25,
  27,
  9,
  42,
  52,
  40,
  26,
  34,
  6,
  35,
  19,
  42,
  37,
  40,
  16,
  38,
  15,
  41,
  16,
  16,
  34,
  11],
 'episode_rewards': [3272,
  595,
  1292,
  2728,
  2241,
  1575,
  3930,
  1927,
  2643,
  2349,
  2756,
  3046,
  3669,
  2947,
  3652,
  2403,
  2615,
  2252,
  3007,
  1079,
  298,
  2244,
  1352,
  885,
  2024,
  2223,
  1411,
  1809,
  1922,
  1400,
  2392,
  2344,
  4107,
  3147,
  1973,
  1146,
  1002,
  3025,
  4333,
  1882,
  2814,
  2904,
  2065,
  3170,
  1938,
  3317,
  

In [13]:
if isinstance(env, FullObsSnakeEnv):
    env.interact = True
else:
    env = ParObsSnakeEnv(grid_size=2*grid_size)
    
state = env.reset()
done = False
while not done:
    action, _ = agent.choose_action(state)
    state, reward, done, _ = env.step(action)
    env.render()
    print(f"Reward: {reward}")

Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: -1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: -1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: -1
Reward: -1
Reward: -1
Reward: -1
Reward: -1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Rewar